In [1]:
#No81(RNNによる予測)
#RNN:中間層で計算した情報を、再び中間層の入力として繰り返し処理を行うという特徴をもつ
import pandas as pd
import re
import numpy as np

# 前処理があまいところがあったので、作成し直す(No50と同じ)
file = '/content/drive/MyDrive/newsCorpora.csv'
data = pd.read_csv(file, encoding='utf-8', header=None, sep='\t', names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])
data = data.replace('"', "'")
# 特定のpublisherのみ抽出
publishers = ['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']
data = data.loc[data['PUBLISHER'].isin(publishers), ['TITLE', 'CATEGORY']].reset_index(drop=True)

# 前処理
def preprocessing(text):
    text_clean = re.sub(r'[\"\'.,:;\(\)#\|\*\+\!\?#$%&/\]\[\{\}]', '', text)
    text_clean = re.sub('[0-9]+', '0', text_clean)
    text_clean = re.sub('\s-\s', ' ', text_clean)
    return text_clean

data['TITLE'] = data['TITLE'].apply(preprocessing)

# 学習用、検証用、評価用に分割する
from sklearn.model_selection import train_test_split

train, valid_test = train_test_split(data, test_size=0.2, shuffle=True, random_state=64, stratify=data['CATEGORY'])
valid, test = train_test_split(valid_test, test_size=0.5, shuffle=True, random_state=64, stratify=valid_test['CATEGORY'])

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

# 単語の頻度
from collections import Counter
words = []
for text in train['TITLE']:
  #空文字で区切って、wordに追加していく
    for word in text.rstrip().split():
        words.append(word)
#数える
c = Counter(words)
#辞書の作成
word2id = {}
for i, cnt in enumerate(c.most_common()):
  ##出現頻度2回以上の単語のみ辞書に追加
    if cnt[1] > 1:
        word2id[cnt[0]] = i + 1
# 出現頻度上位10単語
for i, cnt in enumerate(word2id.items()):
    if i >= 10:
        break
    print(cnt[0], cnt[1])
# 単語のID化
def tokenizer(text):
    words = text.rstrip().split()
#単語辞書からそのwordのidを取得．ない場合は0を返す
    return [word2id.get(word, 0) for word in words]


0 1
to 2
in 3
on 4
UPDATE 5
The 6
as 7
for 8
To 9
of 10


In [3]:
#modelの可視化
!pip install torchinfo
from torch import nn
import random
import torch
from torch import nn
import torch.utils.data as data
from torchinfo import summary
# 乱数のシードを設定
# parserなどで指定
seed = 1234
random.seed(seed)
np.random.seed(seed)
#再現性を保つ
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
#seed値の作成
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(seed)
#RNNクラスの作成
class RNN(nn.Module):
    def __init__(self, vocab_size, emb_size, padding_idx, hidden_size, output_size, num_layers=1):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_size, padding_idx=padding_idx)
        self.rnn = nn.RNN(emb_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    #順伝播
    def forward(self, x, h0=None):
        x = self.emb(x)
        #予測ラベルxと次の隠れ状態
        x, h = self.rnn(x, h0)
        #最後の隠れ状態xの出力層
        x = x[:, -1, :]
        logits = self.fc(x)
        return logits

# パラメータの設定
VOCAB_SIZE = len(set(word2id.values())) + 2  # 辞書のID数 + unknown + パディングID(ID化する単語の種類)
EMB_SIZE = 300 #埋め込みベクトルのサイズ
PADDING_IDX = len(set(word2id.values())) + 1 #各データの単語数を揃えるために空きを埋めるindex
OUTPUT_SIZE = 4 #分類するクラス数
HIDDEN_SIZE = 50 #RNNの隠れ層のサイズ
NUM_LAYERS = 1 #RNNの層数

# モデルの定義
model = RNN(VOCAB_SIZE, EMB_SIZE, PADDING_IDX, HIDDEN_SIZE, OUTPUT_SIZE, NUM_LAYERS)
print(model)

RNN(
  (emb): Embedding(4598, 300, padding_idx=4597)
  (rnn): RNN(300, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=4, bias=True)
)
